<div style="background-color: #ADD8E6; border: 1px solid gray; padding: 3px">
    <h3>Release Planning Sample Workflow</h3>
    The following is an overview of the workflow:
    <ul>
    <li><b>Release Planning</b>: Prepares backlog of issues.</li>
    <li><b>Sprint Planning Cycle</b>: Scopes out sprint (includes Go/No-Go Decision for readiness. If readiness passes, signals kickoff of      next backlog issue, or ENDS if there are no more issues.) Also finetunes sprint tasks by preparing technical documents and plans which will be used as "goldens" for the sprint.</li>
    <li><b>Sprint Implementation Cycle</b>: Iterative implementation of the code,
    documents and/or tests that were indicated/"specified" by the specs in
    the planning phase.</li>
    <li><b>Sprint Evaluation Cycle</b>: Evaluates the code, documents, tests etc.
    created during the implementation cycle.
    </li>
    <li><b>Release Engineering</b>: Prepares the artifacts for readiness review.</li>
    <li><b>Release Delivery</b>: Delivers release.
    </li>
    </ul>
</div>

## Release Initiation / Kickoff

In [1]:
##############################################################################
# Imports
##############################################################################
import importlib
from crewai import Agent, Task, Crew, Process, LLM
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List, Optional
from pydantic import Field, BaseModel
from crewai_tools import SerperDevTool
from crewai.tools import tool
from crewai.flow.flow import Flow, listen, start, router
import os
from urllib.parse import urlparse
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.append(os.path.join(os.path.dirname("__file__"), "..", ".."))
from tools import github_util
import logging
import traceback
import shutil
import time
from templates.crew import ReleaseCycle #, SprintCycle
logging.basicConfig(level=logging.INFO)
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [2]:
##########################################################
# Instance Variables
##########################################################
_GITHUB_PROJECT = os.getenv("GITHUB_PROJECT")

_ORIGINAL_SPEC = os.getenv("ORIGINAL_SPEC")

_FEATURE_BRANCH = "template"

_BASE_BRANCH = "refactored"

_LOCAL_PATH = "tmp"


In [3]:
class ReleaseInitiationFlow(Flow):
    """Flow for ReleaseInitiation."""

    def __init__(self):
        super().__init__()
        
        self.state['retries'] = 0
        
        self.max_depth = 10
    
    @start()
    def setup_environment(self):

        logging.info(f"Starting flow {self.state['id']}...cloning codebase and creating new feature branch...")
        
        github_util.clone_repo(branch=_BASE_BRANCH, local_path=_LOCAL_PATH)
        
        github_util.create_feature_branch(_FEATURE_BRANCH, local_path=_LOCAL_PATH)
        
    
    @listen(setup_environment)
    def start_release_planning(self):

        logging.info(f"Starting flow {self.state['id']}...checking if release has any issues...")

        has_no_issues = github_util.is_project_empty(_GITHUB_PROJECT)

        if has_no_issues:

            logging.info(f"Board has no issues. Creating user stories...")

            additional_context = self.state['additional_context']

            if additional_context:

                additional_context = f"For additional context, read from this single file:\n{additional_context}"
                

            output = ReleaseCycle(_FEATURE_BRANCH, "REFERENCE").crew().kickoff(inputs={"input": self.state['input'],

                                                            "concatenated_output_file": self.state['concatenated_output_file'],

                                                            "additional_context": additional_context,

                                                            "output_base_path": f"{_LOCAL_PATH}/spec"})

            logging.info("Converting user stories into issues...")

            if output.pydantic:

                for story in output.pydantic.stories:
                    
                    github_util.create_issue(title=story.title, 
                                 
                                 body=f"{story.body}\n\n{story.acceptance_criteria}", 
                                 
                                 project=_GITHUB_PROJECT)

            return output


    @listen(start_release_planning)
    def get_sprint_blocked_status(self, spec):

        logging.info(f"Starting flow {self.state['id']}...checking if sprint is blocked...")

        self.state['is_blocked'] = github_util.is_sprint_blocked(_GITHUB_PROJECT)
        
        self.state['is_in_progress'] = github_util.is_sprint_in_progress(_GITHUB_PROJECT)
        

    @listen(get_sprint_blocked_status)
    def get_sprint_next_step(self, result):

        if self.state['is_blocked']:

            return "end"

        if self.state['is_in_progress']:

            self.state['retries'] += 1

            if self.state['retries'] == self.max_depth:

                logging.info("Retries exhausted; exiting.")

                return "end"

            else:

                retries_left = self.max_depth - self.state['retries']

                logging.info(f"Existing issue is being processed. Checking progress again in 60 seconds - {retries_left} tries left...")

                time.sleep(60)

                return self.get_sprint_blocked_status()
                             
        else:

            logging.info("No existing issue is being processed. Processing the next issue in the backlog...")

            return "continue"

                
    @listen("continue")
    def process_next_story(self):

        logging.info("In continue.")

                
    @listen("end")
    def end(self):

        logging.info("Ending workflow.")

### Execute Code Translation Flow
Execute the flow!

In [ ]:
##############################################################################
# Execute the Flow
##############################################################################

flow = ReleaseInitiationFlow()

flow.plot("ReleaseInitiationFlowPlot")

result = flow.kickoff(inputs={"input": f"{_LOCAL_PATH}/docs/document_list.txt",
                              "concatenated_output_file": f"{_LOCAL_PATH}/spec/concatenated.txt",
                              "additional_context": f"{_LOCAL_PATH}/README.md"})

logging.info("Release Initialization flow complete.")

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: ReleaseInitiationFlow                                                                                    │
│  ID: 474c6cd9-9bf0-4e7f-9a96-ae52ceb40f62                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 Flow started with ID: 474c6cd9-9bf0-4e7f-9a96-ae52ceb40f62

INFO:crewai.flow.flow:Flow started with ID: 474c6cd9-9bf0-4e7f-9a96-ae52ceb40f62


/opt/miniconda3/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

INFO:root:Starting flow 474c6cd9-9bf0-4e7f-9a96-ae52ceb40f62...cloning codebase and creating new feature branch...
INFO:root:Cloning repo to tmp...
INFO:root:Repo tmp already exists. Removing...
INFO:root:Repo name: agapebondservant/industrial-data-project


...

06:39:17 - LiteLLM:INFO: utils.py:3258 - 
LiteLLM completion() model= meta-llama/llama-4-maverick; provider = openrouter
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/llama-4-maverick; provider = openrouter
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
06:39:28 - LiteLLM:INFO: utils.py:1260 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Product Manager                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "stories": [                                                                                                 │
│      {                                                                                                          │
│        "title": "Project Requirements Document Creation",                                                       │
│        "body": "As a developer, I want to have a clear understanding of the project requirements so that I can  │
│  develop a comprehensive documentation system that integrates with existing development workflows.",            │
│        "acceptance_criteria": "The document should clearly outline the project's objectives, scope, and         │
│  stakeholders."                                                                                                 │
│      },                                                                                                         │
│      {                                                                                                          │
│        "title": "System Configuration Report Documentation",                                                    │
│        "body": "As a system administrator, I want to understand the default settings for documentation          │
│  generation so that I can configure the system correctly.",                                                     │
│        "acceptance_criteria": "The document should detail the default configuration settings and any            │
│  customization options available."                                                                              │
│      },                                                                                                         │
│      {                                                                                                          │
│        "title": "Architecture Report Documentation",                                                            │
│        "body": "As a developer, I want to understand the microservices design of the architecture so that I     │
│  can develop scalable and flexible components.",                                                                │
│        "acceptance_criteria": "The document should provide an in-depth look at the system's architecture,       │
│  including component interactions and data flows."                                                              │
│      },                                                                                                         │
│      {                                                                                                          │
│        "title": "Software Bill of Materials (SBOM) Documentation",                                              │
│        "body": "As a security auditor, I want to review the SBOM to ensure that all open-source components      │
│  used in the project are up-to-date and secure.",                                                               │
│        "acceptance_criteria": "The SBOM should include a comprehensive list of all open-source components,      │
│  their versions, and any known vulnerabilities."                                                                │
│      },                                                

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: product_owner_task                                                                                       │
│  Agent: Technical Product Manager                                                                               │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9779ec91-67a8-4633-8f30-a8ad454295e9                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "stories": [                                                                                                 │
│      {                                                                                                          │
│        "title": "Project Requirements Document Creation",                                                       │
│        "body": "As a developer, I want to have a clear understanding of the project requirements so that I can  │
│  develop a comprehensive documentation system that integrates with existing development workflows.",            │
│        "acceptance_criteria": "The document should clearly outline the project's objectives, scope, and         │
│  stakeholders."                                                                                                 │
│      },                                                                                                         │
│      {                                                                                                          │
│        "title": "System Configuration Report Documentation",                                                    │
│        "body": "As a system administrator, I want to understand the default settings for documentation          │
│  generation so that I can configure the system correctly.",                                                     │
│        "acceptance_criteria": "The document should detail the default configuration settings and any            │
│  customization options available."                                                                              │
│      },                                                                                                         │
│      {                                                                                                          │
│        "title": "Architecture Report Documentation",                                                            │
│        "body": "As a developer, I want to understand the microservices design of the architecture so that I     │
│  can develop scalable and flexible components.",                                                                │
│        "acceptance_criteria": "The document should provide an in-depth look at the system's architecture,       │
│  including component interactions and data flows."                                                              │
│      },                                                                                                         │
│      {                                                                                                          │
│        "title": "Software Bill of Materials (SBOM) Documentation",                                              │
│        "body": "As a security auditor, I want to review the SBOM to ensure that all open-source components      │
│  used in the project are up-to-date and secure.",                                                               │
│        "acceptance_criteria": "The SBOM should include a comprehensive list of all open-source components,      │
│  their versions, and any known vulnerabilities."      

INFO:root:Converting user stories into issues...
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Issue #40 created: https://github.com/agapebondservant/industrial-data-project/issues/40. Adding issue to project Release 1...
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Adding issue to project 'Release 1'...
INFO:root:Setting issue status to 'Backlog'...
INFO:root:Successfully added issue #40 to project 'Release 1' with status 'Backlog'
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Issue #41 created: https://github.com/agapebondservant/industrial-data-project/issues/41. Adding issue to project Release 1...
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Adding issue to project 'Release 1'...
INFO:root:Setting issue status to 'Backlog'...
INFO:root:Successfully added issue #41 to project 'Release 1' with status 'Backlog'
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:ro

## Planning Cycle

## Implementation Cycle

## Evaluation Cycle

## Readiness Review Cycle

## Sprint Completion Cycle